## Softmax und Cross Entropy Loss 
long version



In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

#### Softmax

Die Sigmoid-Aktivierung nutzen wir, wenn wir zwei Klassen unterscheiden wollen (0,1). Die Softmax-Funktion nutzen wir als Aktivierungsfunktion für den letzten Layer bei mehr als 2 Klassen.


Wird ein neuronales Netz zur Klassifikation benutzt, dann nennt man den letzten Layer mit Neuronen häufig den
**logits-layer**. Er produziert einen Vektor aus reellen Zahlen zwischen minus unendlich und plus unendlich, für jede Klassifikationklasse eine Zahl. Als Aktivierungsfunktion wird häufig die **softmax** -Funktion benutzt. Die Aufgabe der Softmax-Funktion ist es, den Vektor umzuwandeln in einen Vektor aus Wahrscheinlichkeiten dafür, dass die jeweilige Klasse erkannt wurde.


Beispiel: Punkte in der Ebene werden mit drei Farben gefärbt. Ein neuronales Netz soll die Farbe eines Punktes vorhersagen.

Der logits-Layer liefert für einen Input folgende Ausgabe:



In [7]:
logits = torch.tensor([[2, -1, 0.1]])

Je höher die Zahl, desto größer die Wahrscheinlichkeit, dass der Input zu der jeweiligen Klasse gehört.
Die softmax-Funktion rechnet die Werte in Wahrscheinlichkeiten um, d.h. in Zahlen zwischen 0 und 1, die sich zusammen zur 1 addieren.

In [8]:
torch.softmax(logits,dim=1)

tensor([[0.8338, 0.0415, 0.1247]])

Wie werden diese Zahlen berechnet? Da Wahrscheinlichkeiten immer positiv sind, werden zunächst alle drei Zahlen mit der e-Funktion in eine positive Zahl verwandelt.

In [9]:
import math
a = list(logits[0])
for x in a:
    print(math.exp(x))

7.38905609893065
0.36787944117144233
1.1051709197224806


Dann werden die Zahlen noch normiert, damit sie sich zu 1 aufsummieren.

In [11]:
import math
a = list(logits[0]) 
summe = sum([math.exp(x) for x in a])
for x in a:
    print(math.exp(x)/summe)

0.8337810127228958
0.04151151228426165
0.12470747499284246


Wenn wir mehrere Eingaben in einem Batch durch das Netz geschickt haben, enthält der logits-Layer Ausgaben für jedes Batch-Element. Die Softmax-Ausgabe erstellt für jeden Input einen Vektor mit den zugehörigen Wahrscheinlichkeiten. 

In [51]:
logits = torch.tensor([[2, -1, 0.1], [2.5,2,1.8]])
torch.softmax(logits,dim=1)

tensor([[0.8338, 0.0415, 0.1247],
        [0.4755, 0.2884, 0.2361]])

#### Cross Entropy Loss

Um den loss bei einer Klassifikation zu messen wird häufig die cross-entropy-Funktion benutzt. Nehmen wir an, wir berechnen wie oben die Färbung eines Punktes, der drei Farben annehmen kann, die wir mit 0, 1, 2 kodieren. 

In [70]:
target = torch.tensor([2])
logits = torch.tensor([[2,-1,0.1]])
loss = F.cross_entropy(logits, target)
loss

tensor(2.0818)

Wie wird diese Zahl berechnet? Die cross_entropy Funktion wendet zunächst die softmax-Funktion auf die logits an
(dies bedeutet, dass wir in der Definition unseres Netzes die Softmax-Funktion nicht als letzte Aktivierungfunktion
definieren müssen, wenn wir den loss  mit der cross_entropy Funktion messen)


In [71]:
sm = torch.softmax(logits,dim=1)
sm

tensor([[0.8338, 0.0415, 0.1247]])

Dann wird der Target-Wert in einen One-Hot-Vektor umgewandelt. Der One-Hot Vektor hat soviele Komponenten wie es Klassen gibt. Die Stelle mit der richtigen Klasse wird mit einer 1 belegt, alle anderen mit einer 0.

In [79]:
target = torch.tensor([[0,0,1.0]]) 

Dann wird das Skalarprodukt zwischen dem Softmax-Output und dem One-Hot-Vektor gebildet. Das Resultat ist die Wahrscheinlichkeit, die zur 1 gehört.

In [80]:
torch.mm(sm,target.T)

tensor([[0.1247]])

Dann wird der negative Logarithmus dieser Zahl gebildet

In [81]:
- torch.log(torch.mm(sm,target.T))

tensor([[2.0818]])

#### Cross Entropy Intuition

In unserem Beispiel ist der loss $-log(0.1247)$. $0.1247$ ist die Wahrscheinlichkeit, mit der der Algorithmus die korrekte Aussage belegt. Das Ziel des Algorithmus ist es, denn loss zu minimieren. Das geht nur dadurch, dass die Zahl innerhalb des $-log(..)$ größer wird. Die Zahl kann (als Wahrscheinlichkeit) natürlich nicht größer als 1 werden, aber auf alle Fälle wird ein Algorithmus, der versucht, den loss zu minimieren, das dadurch versuchen, diese Zahl der 1 zu nähern. Das bedeutet eine Bewegung in Richtung korrekter Klassifizierung.
